# Linear Regression Consulting Project

In [1]:
import findspark
findspark.init('/home/keegan/spark-2.2.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('cruiseship').getOrCreate()

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [23]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [24]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruisel")
#output = assembler.transform(data)

In [25]:
indexed = indexer.fit(data).transform(data)

In [27]:
assembler = VectorAssembler(
    inputCols=["cruisel","Age", 
               "Tonnage",'passengers',"length","cabins","passenger_density"],
    outputCol="features")

In [28]:
indexed.show()
output = assembler.transform(indexed)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruisel|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|   16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|   16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|    1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|    1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|    1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|    1.0|
|    Elation|   Carnival| 15|            70.367|     20

In [33]:
final_data = output.select("features",'crew')
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [34]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              115|
|   mean| 7.85547826086957|
| stddev|3.576595274907153|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [35]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                43|
|   mean| 7.630232558139536|
| stddev|3.3352514636203066|
|    min|              0.88|
|    max|             12.38|
+-------+------------------+



In [39]:
lr = LinearRegression(labelCol='crew')

In [40]:
lrModel = lr.fit(train_data)

In [41]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [0.0434125460749,-0.00921289955312,0.017285233193,-0.174102952684,0.374038317521,0.868717767154,-0.00614438845032] Intercept: -0.6998075226294587


In [42]:
test_results = lrModel.evaluate(test_data)

In [43]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.6748365334594292
MSE: 0.4554043468915393


In [44]:
test_results.r2

0.9580859941156086